<a href="https://colab.research.google.com/github/joaorafaelm/notebooks/blob/master/Cognitive_complexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title install dependencies
%%capture
!pip install -q cognitive_complexity astunparse tabulate

In [ ]:
#@title imports & utils
import ast
import astunparse
from inspect import getsource
from tabulate import tabulate
from cognitive_complexity.api import get_cognitive_complexity_for_node
from cognitive_complexity.utils.ast import has_recursive_calls, is_decorator, process_child_nodes, process_node_itself

def get_cognitive_complexity(func):
    func = func if isinstance(func, str) else getsource(func)
    funcdef = ast.parse(func).body[0]
    if is_decorator(funcdef):
        return get_cognitive_complexity(funcdef.body[0])

    details = []
    complexity = 0
    for node in funcdef.body:
        node_complexity = get_cognitive_complexity_for_node(node)
        complexity += node_complexity
        node_code = astunparse.unparse(node)
        if f"{funcdef.name}(" in node_code: # +1 for recursion
            node_complexity += 1
            complexity += 1
        details.append([node_complexity, node_code])
    details.append([complexity, "Total"])
    return complexity, details

# Cognitive complexity

In [ ]:
def f(n):
    if n > 10:
        return True
    
    if n < 5:
        return 20
    else:
        return 2
    return f(n)

total, details = get_cognitive_complexity(f)
print(tabulate(details, headers=["Complexity", "Node"], tablefmt="fancy_grid"))

╒══════════════╤═════════════════╕
│   Complexity │ Node            │
╞══════════════╪═════════════════╡
│            1 │ if (n > 10):    │
│              │     return True │
├──────────────┼─────────────────┤
│            2 │ if (n < 5):     │
│              │     return 20   │
│              │ else:           │
│              │     return 2    │
├──────────────┼─────────────────┤
│            1 │ return f(n)     │
├──────────────┼─────────────────┤
│            4 │ Total           │
╘══════════════╧═════════════════╛


# References

* [Cognitive Complexity, Because Testability != Understandability](https://blog.sonarsource.com/cognitive-complexity-because-testability-understandability)

* [Cognitive Complexity: A new way of measuring understandability, white paper by G. Ann Campbell](https://www.sonarsource.com/docs/CognitiveComplexity.pdf)

* [Cognitive Complexity: the New Guide to Refactoring for Maintainable Code](https://www.youtube.com/watch?v=5C6AGTlKSjY)

* [Cognitive Complexity from CodeClimate docs](https://docs.codeclimate.com/docs/cognitive-complexity)

* [Is Your Code Readable By Humans? Cognitive Complexity Tells You](https://www.tomasvotruba.cz/blog/2018/05/21/is-your-code-readable-by-humans-cognitive-complexity-tells-you/)